In [20]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [21]:
%cd nmt-kn-tu/

/content/nmt-kn-tu


In [22]:
! chmod a+x /content/nmt-kn-tu/scripts/data_preparation.sh

In [23]:
! chmod a+x /content/nmt-kn-tu/scripts/split_dataset.sh

In [24]:
! /content/nmt-kn-tu/scripts/data_preparation.sh data.kn data.tu

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 19.44 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 2.60 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 9.90 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads:

In [25]:
! /content/nmt-kn-tu/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.tu

In [26]:
%cd /content/

/content


In [27]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [28]:
!fairseq-preprocess --source-lang kn --target-lang tu \
--trainpref nmt-kn-tu/train/train \
--validpref nmt-kn-tu/dev/dev \
--testpref nmt-kn-tu/test/test \
--destdir nmt-kn-tu/tokenized.kn-tu \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-05 05:23:35 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 05:23:36 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-tu/tokenized.kn-tu', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [29]:
import wandb
wandb.login()
# login authorization.

wandb: Currently logged in as: rahul2023_usa (use `wandb login --relogin` to force relogin)


True

In [33]:
!fairseq-train /content/nmt-kn-tu/tokenized.kn-tu \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "Simple LSTM - kn to tu - 5 March 2022"

2022-03-05 05:24:31 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-05 05:24:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 05:24:35 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'Simple LSTM - kn to tu - 5 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1,

In [34]:
!pwd

/content


In [35]:
!fairseq-generate /content/nmt-kn-tu/tokenized.kn-tu \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-85	-1.5768910646438599	ಕೈ ಮುಗಿದ ್ ಕಲೆವೊಂದು ಸಕಾಯೊಗಾದ ್ ಪಜ ್ ಜೆಗ ್ ಪಜ ್ ಜಿ
P-85	-2.7231 -3.5010 -0.0039 -3.8391 -1.7711 -0.0279 -2.7583 -0.0032 -1.4939 -0.0056 -4.3896 -0.0022 -1.4408 -0.1168
S-116	ತೆಗೆದು ಅದಕ ್ ಕೆ ಅಕ ್ ಕಿ ಹಾಕಿ ಕೈಮುಗಿದು ಹೋಗುತ ್ ತಾರೆ
T-116	ಕಾಯೆರ ್ ಮರತ ಏಳ ್ ಇರೆ
H-116	-1.1217817068099976	ಕಾಯೆರ ್ ಮರತ ಮರತ ಪೊಂಗಾರ ್ ದ ್ ಕಾಯೆರ ್ ರಸ
D-116	-1.1217817068099976	ಕಾಯೆರ ್ ಮರತ ಮರತ ಪೊಂಗಾರ ್ ದ ್ ಕಾಯೆರ ್ ರಸ
P-116	-0.5017 -0.0871 -0.3473 -1.5021 -2.1485 -0.0267 -0.6966 -1.9857 -2.0300 -0.1895 -3.4962 -0.4500
S-213	ಇನ ್ ನು ಕಾಲದಲ ್ ಲಿ ತುಳುನಾಡ ಚರಿತ ್ ರೆಯಲ ್ ಲಿ
T-213	ನನ ಕಾಲೊಡು ತುಲುನಾಡ ಚರಿತ ್ ರೆಡೆ
H-213	-1.816542625427246	ನನ ಕುಡೊರಿಯನ ಇಲ ್ ಲ ್ ಒರಿಯೊಡ ್
D-213	-1.816542625427246	ನನ ಕುಡೊರಿಯನ ಇಲ ್ ಲ ್ ಒರಿಯೊಡ ್
P-213	-0.9286 -2.9674 -1.6067 -3.6293 -0.0001 -2.8410 -1.3641 -4.5803 -0.1189 -0.1290
S-351	ಮಗುವಿನ ತಲೆಗೆ <unk> ಹರಸಿ ಅದಕ ್ ಕೆ <unk> <unk> ಹಾಕುತ ್ ತಾರೆ
T-351	ಬಾಲೆನ ್ ಉಲಾಯಿ <<unk>> ಬೊಕ ್ ಕ
H-351	-1.3961501121520996	ಬಾಲೆಗ ್ ಬತ ್ ತ ್ ಗ ್ ಬತ ್ ತ ್ ಂ